In [ ]:
#!/usr/bin/python
#coding:utf-8
#@ZHOU_YING
#2018-10-31
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import efel
from collections import defaultdict

def ave_voltage_base(voltage_base,sweep_num):
	"求平均voltage_base函数"
	sum_voltage_base=0
	for i in range(0,sweep_num):
		voltage=voltage_base[i]['voltage_base'][0]
		sum_voltage_base=sum_voltage_base+voltage
	average=sum_voltage_base/sweep_num
	return average
def ave_tau(tau,hyper_num):
	"求平均tau函数"
	sum_tau=0
	for i in range(0,hyper_num):
		time_constant=tau[i]['time_constant'][0]
		sum_tau=sum_tau+time_constant
	average=sum_tau/hyper_num
	return average
def Rin_fun(maximum_voltage,stim,sweep_num):
	"求输入阻抗"
	voltage=np.array([])
	for i in range(0,sweep_num):
		voltage=np.append(voltage,maximum_voltage[i]['maximum_voltage'][0])
	slope,_=np.polyfit(stim,voltage,1)
	return slope

def fi_slope(mean_freq,stim,rheobase_index):
	mean_frequency=np.array([])
	for i in range(0,sweep_num-rheobase_index):
		mean_frequency=np.append(mean_frequency,mean_freq[i]['mean_frequency'][0])
	slope,_=np.polyfit(stim[rheobase_index:sweep_num],mean_frequency,1)
	return slope

def ave_sag_amplitude(hyper_num):
	sag_amplitude=efel.getFeatureValues(hyper_sweeps,['sag_amplitude'])
	sum=0
	for i in range(0,hyper_num):
		sag=sag_amplitude[i]['sag_amplitude'][0]
		sum=sum+sag
	average=sum/hyper_num
	return average

def sag_ratio1(hyper_num):
	sag_ratio=efel.getFeatureValues(hyper_sweeps,['sag_ratio1'])
	sum=0
	for i in range(0,hyper_num):
		sag=sag_ratio[i]['sag_ratio1'][0]
		sum=sum+sag
	average=sum/hyper_num
	return average

path='/mnt/f/temp/JSNephysRawdata/'
os.chdir(path)
filelist=pd.read_csv('filelist.csv',header=None)
features=defaultdict(list)
for i in range(0,len(filelist)):
	filename=filelist.loc[i,0]
	features['Id'].append(filename.split(".")[0])
	data=np.loadtxt(filename,skiprows=1)*1000 #导入数据
	time=np.arange(0,1200,float(1200)/len(data))#构建时间向量
	stimulus=pd.DataFrame(data[:,1])
	V=pd.DataFrame(data[:,0])
	length=len(data[0,:])
	index=length/2
	hyper_num=0
	stim=np.array([data[4000,1]])
	#将data中的stim和V数据分开
	for j in range(1,index):
		num_stim=2*j+1
		num_v=2*j
		stimulus.loc[:,j]=pd.DataFrame(data[:,num_stim])
		V.loc[:,j]=pd.DataFrame(data[:,num_v])
		stim_i=data[4000,num_stim]
		unstim_i=data[0,num_stim]
		stim=np.append(stim,data[4000,num_stim])
		if stim_i<unstim_i:
			hyper_num=hyper_num+1
	#构建域上和域下的sweep列表
	sweep_num=V.columns.size
	spike_sweeps=[]
	subthreshold_sweeps=[]
	sweeps=[]
	hyper_sweeps=[]
	for k in range(0,sweep_num):
		sweep={}
		sweep['T']=time
		sweep['V']=V.loc[:,k]
		sweep['stim_start'] = [200]
		sweep['stim_end'] = [1000]
		spikecount = efel.getFeatureValues([sweep], ['Spikecount'])
		sweeps[k:k+1]=[sweep]
		if k<=hyper_num:
			l=len(hyper_sweeps)
			hyper_sweeps[l:l+1]=[sweep]
		else:
			if spikecount[0]['Spikecount'][0]==0:
				a=len(subthreshold_sweeps)
				subthreshold_sweeps[a:a+1]=[sweep]
			else:
				b=len(spike_sweeps)
				spike_sweeps[b:b+1]=[sweep]
	rheobase_sweep=[spike_sweeps[0]]
	#求平均voltage_base
	voltage_base=efel.getFeatureValues(sweeps,['voltage_base']) 
	features['ave_voltage_base'].append(ave_voltage_base(voltage_base,sweep_num)) #平均基线电压
	tau=efel.getFeatureValues(hyper_sweeps,['time_constant'])
	features['tau'].append(ave_tau(tau,hyper_num)) #时间常数
	rheobase_index=len(subthreshold_sweeps)+hyper_num+1
	features['rheobase_i'].append(stimulus.loc[4000,rheobase_index]) #基强度
	voltage_deflection=efel.getFeatureValues(hyper_sweeps,['voltage_deflection']) 
	maximum_voltage=efel.getFeatureValues(sweeps,['maximum_voltage'])
	features['Rin'].append(Rin_fun(maximum_voltage,stim,sweep_num)) #输入阻抗
	mean_freq=efel.getFeatureValues(spike_sweeps,['mean_frequency'])
	features['fi_fit_slope'].append(fi_slope(mean_freq,stim,rheobase_index)) #fi_fit_slope
	features['sag_amplitude'].append(ave_sag_amplitude(hyper_num))
	features['sag_ratio'].append(sag_ratio1(hyper_num))#sag/base-mini
features=pd.DataFrame(features)
features.to_csv('features1.csv',index=False)